In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import linalg as LA

In [5]:
def N0_matrix(I, sequence, trans):
  """
  Creates initial prepared spin system
  Args:
  I (float): nuclear spin
  sequence (string): "fullsat" or "train" depending on pulse sequence used
  trans (int): selected irradiated transition, 0 = (-1/2 <-> 1/2), 1 = (+-3/2 <-> +-1/2), ect.

  Returns:
  N0 (np.array): (2I+1)-dimensional vector describing initial prepared state
  """
  dim = int(2*I) # defines dimension from nuclear spin
  N0 = np.zeros((dim, 1))
  trans_index = int((dim-1)/2) # determines which index the irradiated transition sits at

  # returns an error message if a transition outside the bounds of the nuclear spin is chosen
  if 2*abs(trans) + 1 > dim:
    return print("So such transition for given spin")

  for i in range(dim):
    if sequence == "fullsat":
      if trans_index + trans > 0 and trans_index + trans < dim:
        N0[trans_index + trans, 0] = -2

      if trans_index + trans - 1 > 0:
        N0[trans_index + trans - 1, 0] = 1

      if trans_index + trans + 1 < dim:
        N0[trans_index + trans + 1, 0] = 1

    if sequence == "train":
      if trans_index + trans > 0 and trans_index + trans < dim:
        N0[trans_index + trans, 0] = -1

  return N0

In [2]:
##### Define measurement parameters #####

# nuclear spin of isotope probed
I = 9/2

# selected irradiated transition (0 = central (-1/2 <-> 1/2), 1 = 1st sat (+-3/2 <-> +-1/2), ect.)
trans = 1

# type of pulse sequence used, can be "fullsat" or "train"
sequence = "fullsat" # "train"

In [14]:
# Define initial prepared state
N0 = N0_matrix(I, sequence, trans)
print("N0 = ", N0)

N0 =  [[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [-2.]
 [ 1.]
 [ 0.]
 [ 0.]]


In [3]:
####### reduced recovery matrix #######
# must be manually added

if I == 3/2:
  R = np.array([[-6, 4, 0],
      [3, -8, 3],
      [0, 4, -6]])

if I == 5/2:
  R = np.array([[-10, 8, 0, 0, 0],
                [5, -16, 9, 0, 0],
                [0, 8, -18, 8, 0],
                [0, 0, 9, -16, 5],
                [0, 0, 0, 8, -10]])

if I == 9/2:
  R = np.array([[-18, 16, 0, 0, 0, 0, 0, 0, 0],
                [9, -32, 21, 0, 0, 0, 0, 0, 0],
                [0, 16, -42, 24, 0, 0, 0, 0, 0],
                [0, 0, 21, -48, 25, 0, 0, 0, 0],
                [0, 0, 0, 24, -50, 24, 0, 0, 0],
                [0, 0, 0, 0, 25, -48, 21, 0, 0],
                [0, 0, 0, 0, 0, 24, -42, 16, 0],
                [0, 0, 0, 0, 0, 0, 21, -32, 9],
                [0, 0, 0, 0, 0, 0, 0, 16, -18]])
print("R = ", R)

[[-18  16   0   0   0   0   0   0   0]
 [  9 -32  21   0   0   0   0   0   0]
 [  0  16 -42  24   0   0   0   0   0]
 [  0   0  21 -48  25   0   0   0   0]
 [  0   0   0  24 -50  24   0   0   0]
 [  0   0   0   0  25 -48  21   0   0]
 [  0   0   0   0   0  24 -42  16   0]
 [  0   0   0   0   0   0  21 -32   9]
 [  0   0   0   0   0   0   0  16 -18]]


In [4]:
#calculate eigenvalues and eigenvectors of R
L, ET = LA.eig(R)
print("eigenvalues = ", L)
# returns transpose of E as defined in literature instead of E

# get transpose of eigenvector matrix
E = ET.transpose()

# Take inverse of ET
invET = LA.inv(ET)

eigenvalues =  [-90. -72. -56. -42. -30. -20.  -2.  -6. -12.]


In [10]:
# calculate prefactors for relaxation
ETinvN0 = np.matmul(invET, N0)

# chooses the correct row in E for the irradiated transition
dim = int(2*I)
n = int((trans + dim)/2)
trex = np.zeros((dim, 1))
E_trans = E[:, n]

# takes the transpose for matrix multiplication purposes
for i in range(dim):
  trex[i, 0] = E_trans[i]

# calculates ai
ai = np.multiply(ETinvN0, trex)

In [9]:
listofai = []

# rounds ai to 3 decimal points and appends to list
for i in ai:
  listofai.append(np.round(*i,4))

# print final magnetization recovery curve
print("M(t) = 1")

for i in range(len(listofai)):
  if listofai[i] == 0:
    i+=1
  else:
    print(listofai[i],"exp(",np.round(L[i]/2,0),"t/T1)")

M(t) = 1
-0.907 exp( -45.0 t/T1)
-0.6168 exp( -36.0 t/T1)
-0.0067 exp( -28.0 t/T1)
-0.297 exp( -21.0 t/T1)
-0.0321 exp( -15.0 t/T1)
-0.0787 exp( -10.0 t/T1)
-0.0121 exp( -1.0 t/T1)
-0.0076 exp( -3.0 t/T1)
-0.0421 exp( -6.0 t/T1)
